In [4]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import random

In [6]:
current_dir = os.getcwd()
CLEAN_DIR = os.path.join(current_dir, "content", "data", "clean")
NOISY_BASE_DIR = os.path.join(current_dir, "content", "data", "noisy")
os.makedirs(CLEAN_DIR, exist_ok=True)
os.makedirs(NOISY_BASE_DIR, exist_ok=True)

In [8]:
GAUSSIAN_DIR = os.path.join(NOISY_BASE_DIR, "gaussian")
SALT_PEPPER_DIR = os.path.join(NOISY_BASE_DIR, "salt_pepper")
POISSON_DIR = os.path.join(NOISY_BASE_DIR, "poisson")
SPECKLE_DIR = os.path.join(NOISY_BASE_DIR, "speckle")
UNIFORM_DIR = os.path.join(NOISY_BASE_DIR, "uniform")

In [9]:
GAUSSIAN_SALT_PEPPER_DIR = os.path.join(NOISY_BASE_DIR, "gaussian_salt_pepper")
GAUSSIAN_POISSON_DIR = os.path.join(NOISY_BASE_DIR, "gaussian_poisson")
GAUSSIAN_SPECKLE_DIR = os.path.join(NOISY_BASE_DIR, "gaussian_speckle")
GAUSSIAN_UNIFORM_DIR = os.path.join(NOISY_BASE_DIR, "gaussian_uniform")
SALT_PEPPER_SPECKLE_DIR = os.path.join(NOISY_BASE_DIR, "salt_pepper_speckle")
SALT_PEPPER_UNIFORM_DIR = os.path.join(NOISY_BASE_DIR, "salt_pepper_uniform")
POISSON_SPECKLE_DIR = os.path.join(NOISY_BASE_DIR, "poisson_speckle")
POISSON_UNIFORM_DIR = os.path.join(NOISY_BASE_DIR, "poisson_uniform")
SPECKLE_UNIFORM_DIR = os.path.join(NOISY_BASE_DIR, "speckle_uniform")

In [10]:
GAUSSIAN_SALT_PEPPER_SPECKLE_DIR = os.path.join(NOISY_BASE_DIR, "gaussian_salt_pepper_speckle")
GAUSSIAN_POISSON_UNIFORM_DIR = os.path.join(NOISY_BASE_DIR, "gaussian_poisson_uniform")
SALT_PEPPER_SPECKLE_UNIFORM_DIR = os.path.join(NOISY_BASE_DIR, "salt_pepper_speckle_uniform")

In [11]:
noise_dirs = [
    GAUSSIAN_DIR, SALT_PEPPER_DIR, POISSON_DIR, SPECKLE_DIR, UNIFORM_DIR,
    GAUSSIAN_SALT_PEPPER_DIR, GAUSSIAN_POISSON_DIR, GAUSSIAN_SPECKLE_DIR, GAUSSIAN_UNIFORM_DIR,
    SALT_PEPPER_SPECKLE_DIR, SALT_PEPPER_UNIFORM_DIR, POISSON_SPECKLE_DIR, POISSON_UNIFORM_DIR,
    SPECKLE_UNIFORM_DIR, GAUSSIAN_SALT_PEPPER_SPECKLE_DIR, GAUSSIAN_POISSON_UNIFORM_DIR,
    SALT_PEPPER_SPECKLE_UNIFORM_DIR
]

In [12]:
for dir_path in noise_dirs:
    os.makedirs(dir_path, exist_ok=True)


In [13]:
def add_gaussian_noise(image_array, mean=0, std=25):
    """Add Gaussian noise to image"""
    noise = np.random.normal(mean, std, image_array.shape)
    noisy_image = image_array + noise
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

def add_salt_pepper_noise(image_array, salt_prob=0.05, pepper_prob=0.05):
    """Add salt and pepper noise to image"""
    noisy_image = image_array.copy()

    # Salt noise (white pixels)
    salt_mask = np.random.random(image_array.shape[:2]) < salt_prob
    noisy_image[salt_mask] = 255

    # Pepper noise (black pixels)
    pepper_mask = np.random.random(image_array.shape[:2]) < pepper_prob
    noisy_image[pepper_mask] = 0

    return noisy_image

def add_poisson_noise(image_array):
    """Add Poisson noise to image"""
    # Convert to float, add poisson noise, then convert back
    image_float = image_array.astype(np.float32)
    noisy_image = np.random.poisson(image_float / 255.0 * 50) / 50.0 * 255.0
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

def add_speckle_noise(image_array, variance=0.1):
    """Add speckle noise to image"""
    noise = np.random.normal(0, variance, image_array.shape)
    noisy_image = image_array + image_array * noise
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

def add_uniform_noise(image_array, low=-20, high=20):
    """Add uniform noise to image"""
    noise = np.random.uniform(low, high, image_array.shape)
    noisy_image = image_array + noise
    return np.clip(noisy_image, 0, 255).astype(np.uint8)


In [14]:
def process_images_with_noise():
    """Process all clean images and add different types of noise"""

    # Get all clean images
    clean_images = [f for f in os.listdir(CLEAN_DIR) if f.endswith('.png')]
    print(f"Found {len(clean_images)} clean images to process")

    for img_name in tqdm(clean_images, desc="Processing images"):
        img_path = os.path.join(CLEAN_DIR, img_name)

        try:
            # Load image
            img = Image.open(img_path).convert("RGB")
            img_array = np.array(img)

            # Apply individual noise types
            gaussian_noisy = add_gaussian_noise(img_array)
            salt_pepper_noisy = add_salt_pepper_noise(img_array)
            poisson_noisy = add_poisson_noise(img_array)
            speckle_noisy = add_speckle_noise(img_array)
            uniform_noisy = add_uniform_noise(img_array)

            # Apply combination noise types
            gaussian_salt_pepper = add_salt_pepper_noise(add_gaussian_noise(img_array))
            gaussian_poisson = add_poisson_noise(add_gaussian_noise(img_array))
            gaussian_speckle = add_speckle_noise(add_gaussian_noise(img_array))
            gaussian_uniform = add_uniform_noise(add_gaussian_noise(img_array))
            salt_pepper_speckle = add_speckle_noise(add_salt_pepper_noise(img_array))
            salt_pepper_uniform = add_uniform_noise(add_salt_pepper_noise(img_array))
            poisson_speckle = add_speckle_noise(add_poisson_noise(img_array))
            poisson_uniform = add_uniform_noise(add_poisson_noise(img_array))
            speckle_uniform = add_uniform_noise(add_speckle_noise(img_array))

            # Apply triple combinations
            gaussian_salt_pepper_speckle = add_speckle_noise(add_salt_pepper_noise(add_gaussian_noise(img_array)))
            gaussian_poisson_uniform = add_uniform_noise(add_poisson_noise(add_gaussian_noise(img_array)))
            salt_pepper_speckle_uniform = add_uniform_noise(add_speckle_noise(add_salt_pepper_noise(img_array)))

            # Save individual noise types
            Image.fromarray(gaussian_noisy).save(os.path.join(GAUSSIAN_DIR, img_name))
            Image.fromarray(salt_pepper_noisy).save(os.path.join(SALT_PEPPER_DIR, img_name))
            Image.fromarray(poisson_noisy).save(os.path.join(POISSON_DIR, img_name))
            Image.fromarray(speckle_noisy).save(os.path.join(SPECKLE_DIR, img_name))
            Image.fromarray(uniform_noisy).save(os.path.join(UNIFORM_DIR, img_name))

            # Save combination noise types
            Image.fromarray(gaussian_salt_pepper).save(os.path.join(GAUSSIAN_SALT_PEPPER_DIR, img_name))
            Image.fromarray(gaussian_poisson).save(os.path.join(GAUSSIAN_POISSON_DIR, img_name))
            Image.fromarray(gaussian_speckle).save(os.path.join(GAUSSIAN_SPECKLE_DIR, img_name))
            Image.fromarray(gaussian_uniform).save(os.path.join(GAUSSIAN_UNIFORM_DIR, img_name))
            Image.fromarray(salt_pepper_speckle).save(os.path.join(SALT_PEPPER_SPECKLE_DIR, img_name))
            Image.fromarray(salt_pepper_uniform).save(os.path.join(SALT_PEPPER_UNIFORM_DIR, img_name))
            Image.fromarray(poisson_speckle).save(os.path.join(POISSON_SPECKLE_DIR, img_name))
            Image.fromarray(poisson_uniform).save(os.path.join(POISSON_UNIFORM_DIR, img_name))
            Image.fromarray(speckle_uniform).save(os.path.join(SPECKLE_UNIFORM_DIR, img_name))

            # Save triple combinations
            Image.fromarray(gaussian_salt_pepper_speckle).save(os.path.join(GAUSSIAN_SALT_PEPPER_SPECKLE_DIR, img_name))
            Image.fromarray(gaussian_poisson_uniform).save(os.path.join(GAUSSIAN_POISSON_UNIFORM_DIR, img_name))
            Image.fromarray(salt_pepper_speckle_uniform).save(os.path.join(SALT_PEPPER_SPECKLE_UNIFORM_DIR, img_name))

        except Exception as e:
            print(f"Failed to process {img_name}: {e}")

    print(f"\n----Noise addition completed!")
    print(f"Clean images: {len(clean_images)}")
    print(f"Noise directories created: {len(noise_dirs)}")
    print(f"Total noisy images generated: {len(clean_images) * len(noise_dirs)}")

In [15]:
process_images_with_noise()

Found 3009 clean images to process


Processing images: 100%|██████████| 3009/3009 [17:45<00:00,  2.82it/s]


----Noise addition completed!
Clean images: 3009
Noise directories created: 17
Total noisy images generated: 51153
